In [7]:
# Disable info messages from the tensorflow
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
################## Import the libraries #####################
import numpy as np
import pandas as pd
import keras
from keras.models import Model, Sequential
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D,MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler,scale
from sklearn.metrics import accuracy_score
from scipy.ndimage import zoom
import pickle

In [8]:

#Load train data
with open('train', 'rb') as fo:         #Open file
    d = pickle.load(fo, encoding='bytes')       #Load pickle data
    train_data = d['data']                      #Load train data
    train_label = d['label']                    #Load train label
 
 
with open('test', 'rb') as fo:          #Open file
    d = pickle.load(fo, encoding='bytes')       #Load pickle data
    test_data = d['data']                       #Load test data
    test_label = d['label']                     #Load test data
#print(train_data.shape,test_data.shape) # (84420,310) # (58128,310)
#Merge train and test data in order to scale them
X = np.concatenate([train_data,test_data])
# scaling the inputs from 0 to 1
X = scale(X)
Y = np.concatenate([train_label,test_label])
classes = len(np.unique(Y))
Y = LabelEncoder().fit_transform(Y)
Y = to_categorical(Y,classes)
print(X.shape,Y.shape) #(142548,310)

(142548, 310) (142548, 3)


In [9]:
# split into train and test data
X_train = X[:84420]
X_test =  X[84420:]
Y_train = Y[:84420]
Y_test = Y[84420:]

In [10]:
n = X_train.shape[1]
dim = X_train.shape[1]
final_dim = min(32*32,n*n)
# convert 2-D to 3-D.
while dim<final_dim:
  features = int(dim/2)
  #f = feature-60
  model = Sequential()
  if (dim+features)>final_dim:
    features = final_dim - dim
    model.add(Dense(features,name='feature', activation='relu',input_shape=(dim,)))
  else:
    model.add(Dense(features,name='feature', activation='relu',input_shape=(dim,)))
  model.add(Dense(classes,activation='softmax'))
  model.compile(loss='categorical_crossentropy',optimizer='adam')
  model.fit(X_train,Y_train,epochs=1,batch_size=512,verbose=1)
  extract = Model(model.inputs, model.get_layer('feature').output)
  features1 = extract.predict(X_train)
  features2 = extract.predict(X_test)
  X_train = np.concatenate((X_train, features1), axis=1) # concatenate on horizontal axis
  X_test = np.concatenate((X_test, features2), axis=1)
  dim = X_train.shape[1]
print(X_train.shape,X_test.shape)
s = MinMaxScaler()
X_train = s.fit_transform(X_train)
X_test = s.transform(X_test)

165/165 [==============================] - 0s 2ms/step - loss: 0.0483
(84420, 1024) (58128, 1024)


In [11]:
from PIL import Image
x_train = []
x_test = []
for i in range(len(X_train)):
    img = Image.fromarray(X_train[i])
    imgs = img.resize(size=(32, 32))
    imgs = np.array(imgs)
    x_train.append(np.repeat(imgs[:, :, np.newaxis], 3, axis=2))
x_train = np.array(x_train,dtype='float32')
for i in range(len(X_test)):
    img = Image.fromarray(X_test[i])
    imgs = img.resize(size=(32, 32))
    imgs = np.array(imgs)
    x_test.append(np.repeat(imgs[:, :, np.newaxis], 3, axis=2))
x_test = np.array(x_test,dtype='float32')
print(x_train.shape,x_test.shape)

(84420, 32, 32, 3) (58128, 32, 32, 3)


In [12]:
############### Feature-extraction ###############
# pretrained model for FE
pretrained_model = keras.applications.DenseNet121(input_shape=(32,32,3), weights=None, include_top=False)
x = Flatten()(pretrained_model.output)
x = Dense(512, activation='relu',name='flattenn')(x)
x = Dropout(0.5)(x)
prediction = Dense(classes, activation='softmax')(x)
model = Model(inputs=pretrained_model.input, outputs=prediction)
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)
# fit training data and validate on the testing data
model.fit(x_train,Y_train,batch_size=256, epochs=10,verbose=1,validation_data=(x_test,Y_test))

Epoch 1/10
330/330 [==============================] - 32s 96ms/step - loss: 0.0911 - accuracy: 0.9664 - val_loss: 6.3986 - val_accuracy: 0.3432
Epoch 2/10
330/330 [==============================] - 25s 77ms/step - loss: 0.0147 - accuracy: 0.9955 - val_loss: 2.0017 - val_accuracy: 0.6700
Epoch 3/10
330/330 [==============================] - 25s 77ms/step - loss: 0.0077 - accuracy: 0.9977 - val_loss: 2.2267 - val_accuracy: 0.7487
Epoch 4/10
330/330 [==============================] - 26s 77ms/step - loss: 0.0085 - accuracy: 0.9974 - val_loss: 1.9531 - val_accuracy: 0.7424
Epoch 5/10
330/330 [==============================] - 26s 78ms/step - loss: 0.0075 - accuracy: 0.9978 - val_loss: 2.4423 - val_accuracy: 0.7545
Epoch 6/10
330/330 [==============================] - 26s 78ms/step - loss: 4.6234e-04 - accuracy: 0.9999 - val_loss: 2.7921 - val_accuracy: 0.7571
Epoch 7/10
330/330 [==============================] - 26s 78ms/step - loss: 1.1761e-05 - accuracy: 1.0000 - val_loss: 2.8915 - val_a

In [13]:
score = model.evaluate(x_test,Y_test)
acc = model.evaluate(x_train,Y_train)
print('training accuracy: ',acc[1])
print('testing accuracy: ',score[1])

2639/2639 [==============================] - 29s 11ms/step - loss: 2.1526e-07 - accuracy: 1.0000
training accuracy:  1.0
testing accuracy:  0.7581028342247009
